In [47]:
import pandas as pd

# Read the CSV file into a DataFrame
recent_rent = pd.read_csv('../data/landing/2024_rent.csv')
history_rent = pd.read_csv('../data/landing/history_rent.csv')
postcode = pd.read_csv('../data/raw/external/australian_postcodes.csv')

# Merge historical data and recent data together

In [48]:
# Splitting the 'Location' column by '-' and expanding it into multiple rows
history_rent = history_rent.assign(Location=history_rent['Location'].str.split('-')).explode('Location').reset_index(drop=True)

In [49]:
history_rent = history_rent.drop(columns=['Zone'])
history_rent.head(3)

,Location,Year,Month,Count,Median,Bed,Apartment
0,Albert Park,2000,Dec,369,175,1,1
1,Middle Park,2000,Dec,369,175,1,1
2,West St Kilda,2000,Dec,369,175,1,1


In [50]:
# Concatenate the two DataFrames along rows (axis=0)
rent = pd.concat([history_rent, recent_rent], ignore_index=True)

# Convert the 'Median' column to numeric (handle errors), round floats, fill NaN with a default value, then convert to integer
rent['Median'] = pd.to_numeric(rent['Median'], errors='coerce').round().fillna(0).astype(int)


# Rename Location into Suburb
rent = rent.rename(columns={'Location':'Suburb'})

# Convert 'locality' to lowercase and rename to 'Suburb'
rent.loc[:,'Suburb'] = rent['Suburb'].str.lower()

# Replace 'mt' with 'mountain' in the 'Suburb' column
rent['Suburb'] = rent['Suburb'].str.replace(r'\bmt\b', 'mount', regex=True)

# Replace 'newcombe' with 'newcomb' in the 'Suburb' column
rent['Suburb'] = rent['Suburb'].str.replace('newcombe', 'newcomb')

# Replace 'wanagaratta' with 'newcomb' in the 'Suburb' column
rent['Suburb'] = rent['Suburb'].str.replace('wanagaratta', 'wangaratta')

# Replace 'gladstone st south melbourne' with 'south melbourne' in the 'Suburb' column
rent['Suburb'] = rent['Suburb'].str.replace('gladstone st south melbourne', 'south melbourne')
rent.head(3)


,Suburb,Year,Month,Count,Median,Bed,Apartment
0,albert park,2000,Dec,369,175,1,1
1,middle park,2000,Dec,369,175,1,1
2,west st kilda,2000,Dec,369,175,1,1


In [51]:
# Filter the DataFrame to keep only rows where 'State' is 'VIC'
postcode = postcode[postcode['state'] == 'VIC']
# Convert 'locality' to lowercase and rename to 'Suburb'
postcode.loc[:,'locality'] = postcode['locality'].str.lower()
postcode.rename(columns={'locality': 'Suburb'}, inplace=True)
postcode.head(3)

,id,postcode,Suburb,state,long,lat,dc,type,status,sa3,...,altitude,chargezone,phn_code,phn_name,lgaregion,lgacode,electorate,electoraterating,sed_code,sed_name
6202,4746,3000,melbourne,VIC,144.982585,-37.814437,CITY DELIVERY CENTRE,Delivery Area,Updated 17-Mar-2024 AUSPOST,20604.0,...,27.332188,V1,PHN201,North Western Melbourne,Melbourne,24600.0,Melbourne,Inner Metropolitan,24703.0,Melbourne (Northern Metropolitan)
6203,4747,3001,melbourne,VIC,144.982585,-37.814437,CITY MAIL PROCESSING CENTRE,Post Office Boxes,Updated 17-Mar-2024 AUSPOST,20605.0,...,27.332188,V1,PHN203,NaN,Melbourne,24600.0,Maribyrnong,NaN,NaN,NaN
6204,4748,3002,east melbourne,VIC,144.982585,-37.814437,CITY DELIVERY CENTRE,Delivery Area,Updated 17-Mar-2024 AUSPOST,20604.0,...,27.332188,V1,PHN201,North Western Melbourne,Melbourne,24600.0,Melbourne,Inner Metropolitan,24703.0,Melbourne (Northern Metropolitan)


In [52]:
import re


# Function to move direction (e.g., "West", "East", etc.) to the beginning of the suburb name
def move_direction_to_start(suburb):
    # Define a regex pattern to find direction words (e.g., "West", "East", "North", "South")
    direction_pattern = r'\b(west|east|north|south)\b'
    
    # Search for direction words at the end of the suburb name
    match = re.search(direction_pattern, suburb)
    
    if match:
        direction = match.group(0)
        # Remove the direction from the end and place it at the start
        suburb_without_direction = re.sub(direction_pattern, '', suburb).strip()
        return f'{direction} {suburb_without_direction}'
    
    return suburb

# Apply the function to clean suburb names in both datasets
rent['Cleaned_Suburb'] = rent['Suburb'].apply(move_direction_to_start)
postcode['Cleaned_Suburb'] = postcode['Suburb'].apply(move_direction_to_start)

# Perform the merge using the cleaned suburb names
rent = pd.merge(rent, postcode[['Cleaned_Suburb', 'postcode']], on='Cleaned_Suburb', how='left')

# Drop the helper column 'Cleaned_Suburb' if needed
rent = rent.drop(columns=['Cleaned_Suburb'])


# Display the merged data
print(rent.head())

          Suburb  Year Month  Count  Median  Bed  Apartment  postcode
0    albert park  2000   Dec    369     175    1          1    3206.0
1    middle park  2000   Dec    369     175    1          1    3206.0
2  west st kilda  2000   Dec    369     175    1          1    3182.0
3    albert park  2000   Jun    347     165    1          1    3206.0
4    middle park  2000   Jun    347     165    1          1    3206.0


In [53]:
# Replace the postcode with '3030' for the suburb 'sanctuary lakes'
rent.loc[rent['Suburb'] == 'sanctuary lakes', 'postcode'] = '3030'

# Replace the postcode with '3004' for the suburb 'cbd' and 'st kilda rd'
rent.loc[rent['Suburb'] == 'cbd', 'postcode'] = '3004'
rent.loc[rent['Suburb'] == 'st kilda rd', 'postcode'] = '3004'

# Replace the postcode with '3132' for the suburb 'yarra ranges'
rent.loc[rent['Suburb'] == 'yarra ranges', 'postcode'] = '3132'


/var/folders/4f/qwqd2cfs1674nsc2kmx8hhp00000gn/T/ipykernel_2660/4239275503.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3030' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  rent.loc[rent['Suburb'] == 'sanctuary lakes', 'postcode'] = '3030'


In [54]:
# Convert 'postcode' to integer
rent['postcode'] = pd.to_numeric(rent['postcode'], errors='coerce').astype('Int64')

# Drop 'Suburb' and 'Count' columns
rent = rent.drop(columns=['Suburb', 'Count'])

# Remove duplicates
rent = rent.drop_duplicates()
rent.head(3)

,Year,Month,Median,Bed,Apartment,postcode
0,2000,Dec,175,1,1,3206
2,2000,Dec,175,1,1,3182
3,2000,Jun,165,1,1,3206


# Merge rent data with external data (park, school, transportation, population and income)

In [55]:
park = pd.read_csv('../data/landing/parks.csv')
# Rename POSTCODE into postcode
park = park.rename(columns={'POSTCODE':'postcode'})
# Merge the dataframes on the postcode
rent = pd.merge(rent, park, on='postcode', how='inner') 

In [56]:
school_recent = pd.read_csv('../data/landing/schoolscount2023.csv')
school_past = pd.read_csv('../data/landing/schoolscount2018.csv')

rent_recent = rent[rent['Year'] > 2018]
rent_past = rent[rent['Year'] < 2019]

# Merge the dataframes on the postcode
rent_recent = pd.merge(rent_recent, school_recent, on='postcode', how='inner') 
rent_past = pd.merge(rent_past, school_past, on='postcode', how='inner') 

# Concatenate the two DataFrames along rows (axis=0)
rent = pd.concat([rent_recent, rent_past], ignore_index=True)

rent

,Year,Month,Median,Bed,Apartment,postcode,park_count,mean_park_area,Camp,Language,Primary,Secondary,Special,school_total
0,2019,Dec,370,1,1,3182,1,218.494,0,0,4,3,0,7
1,2019,Jun,350,1,1,3182,1,218.494,0,0,4,3,0,7
2,2019,Mar,350,1,1,3182,1,218.494,0,0,4,3,0,7
3,2019,Sep,360,1,1,3182,1,218.494,0,0,4,3,0,7
4,2020,Dec,350,1,1,3182,1,218.494,0,0,4,3,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49468,2017,Sep,410,4,0,3036,8,192.473,0,0,3,2,0,5
49469,2018,Dec,500,4,0,3036,8,192.473,0,0,3,2,0,5
49470,2018,Jun,460,4,0,3036,8,192.473,0,0,3,2,0,5
49471,2018,Mar,420,4,0,3036,8,192.473,0,0,3,2,0,5


In [57]:
transportation = pd.read_csv('../data/landing/transportation.csv')
income = pd.read_csv('../data/landing/income_2000-27.csv')
income = income.rename(columns={'Postcode': 'postcode'})
population = pd.read_csv('../data/landing/population_2000-27.csv')
population = population.rename(columns={'Postcode': 'postcode'})

# Merge the dataframes on the postcode
rent = pd.merge(rent, transportation, on='postcode', how='inner') 
rent = pd.merge(rent, income, on='postcode', how='inner') 
rent = pd.merge(rent, population, on='postcode', how='inner') 



In [58]:
rent

,Year,Month,Median,Bed,Apartment,postcode,park_count,mean_park_area,Camp,Language,...,2018_y,2019_y,2020_y,2021_y,2022_y,2023_y,2024_y,2025_y,2026_y,2027_y
0,2019,Dec,370,1,1,3182,1,218.4940,0,0,...,29239,29443,29028,26176,26538,28666,31004,31588,32182,32787
1,2019,Jun,350,1,1,3182,1,218.4940,0,0,...,29239,29443,29028,26176,26538,28666,31004,31588,32182,32787
2,2019,Mar,350,1,1,3182,1,218.4940,0,0,...,29239,29443,29028,26176,26538,28666,31004,31588,32182,32787
3,2019,Sep,360,1,1,3182,1,218.4940,0,0,...,29239,29443,29028,26176,26538,28666,31004,31588,32182,32787
4,2020,Dec,350,1,1,3182,1,218.4940,0,0,...,29239,29443,29028,26176,26538,28666,31004,31588,32182,32787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29321,2024,Aug,1000,4,0,3796,6,1275.9650,0,0,...,10086,10059,9949,9866,9807,9919,10168,10207,10247,10288
29322,2024,Aug,582,4,0,3752,5,745.4096,0,0,...,25719,25821,25798,25227,25114,25467,40203,43016,46026,49246
29323,2024,Aug,872,4,0,3107,2,471.7175,0,0,...,14419,14441,14581,14280,14346,14445,14602,14657,14712,14768
29324,2024,Aug,500,3,0,3750,1,1290.2160,0,0,...,14544,17309,20975,24807,28244,30758,49244,63781,82609,106996


In [59]:
file_path = '../data/curated/rent.csv'

# Save the DataFrame to the specified path in CSV format
rent.to_csv(file_path, index=False)